<a href="https://colab.research.google.com/github/ChristianKitte/HelloKNN/blob/main/KNN_Algorithmus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Die eigentliche Lösung der Aufgabe ist ganz am Ende zu finden...**

Ein interssanter Link zum Thema Datenvorbereitung:


https://cloud.google.com/architecture/data-preprocessing-for-ml-with-tf-transform-pt1?hl=de


---

Der erste Teil der anstehenden Aufgabe zum Thema instanzabsiertes Lernen ist die Programmierung eines eigenen Logarithmus hierzu. Teile des folgenden Codes beinhalten Alternativen mit Sklearn oder Tests, um Vergleiche mit meiner eigenen Lösung anzustellen. 

Diese wurden in der Abgabe belassen, da diese auch für mich selbst als NAchschlagewerk dient.


---

In einem ersten Schritt erfolgen alle notwendigen Importe für die spätere Nutzung:

In [ ]:
# Initialize

import numpy as np
import pandas as pd
import math

#from scipy import stats

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plot

print("Import wurde ausgeführt....")

Import wurde ausgeführt....


Um über eigene Adressen zu verfügen, wird anschließend eine kleine Adressbasis definiert.

In [ ]:
#Testdaten

testdaten=[1,4,2,3426,345,44,3453,7]
#testdaten=[1,2,3,4,5,6]
#testdaten=[1000,765,800]

print("Testdaten erzeugt...")

Testdaten erzeugt...


Als Basis dieser Übung dient der auch schon in der Übung zur Regression eingesetzte eigene CSV Datensatz. Im fiktiven Kontext der Nutzung wurde beschlossen, aus den Zahlen der Zuteilungen und erfolgreihen Bearbeitung (siehe hierzu die Ausführungen im Rahmen der Aufgabe zur Regression) einen Quotionen zu berechnen und darauf basierend die Mitarbeiter zu klassifizieren. 

Es wurden vier Klassen auf Basis der vorliegenden "Basisdaten" definiert:
* 0 unbestimmt      unbestimmt
* 1 Top             =1
* 2 Best            >1
* 3 Score           >=1.2
* 4 Underscore      >=2

(Hierbei ist es natürlich diskussionswürdig, ob diese Art zum einen das Verhältnis belastet, zum anderen in dieser Einfachheit sinnvoll ist. Stichwort feature enginiering)


---

In der folgenden Zelle werden die erweiterten Daten aus GitHub geholt und aufgeteilt. Sie werden nicht gesplittet und ndurcheinander gemischt, da dieser Algorithmus grundsätzlcih alle Daten nutzt und determinstisch/nachvollziehbar ist.

In [ ]:
# Daten von GitHub besorgen und Überschriften zur besseren Übersicht anpassen

ds=pd.read_csv("https://raw.githubusercontent.com/ChristianKitte/HelloKNN/main/Auftraege.csv")
ds=ds.rename(columns={"Anzahl angenommene Aufträge":"Aufträge","Anzahl erfolgreiche Ausführung":"erfolgreich", "Angenommen durch Erfolg":"Faktor", "Klasse":"Label"})

# Es wurden vier Klassen auf Basis der vorliegenden "Basisdaten" definiert:
# 0 unbestimmt      unbestimmt
# 1 Top             =1
# 2 Best            >1
# 3 Score           >=1.2
# 4 Underscore      >=2

# X (Aufträge), Y (Anzahl erfolgreich), labels (Klassen)
x = ds.iloc[:,0].values
y = ds.iloc[:,1].values
data=ds.iloc[:,:2].values # Col 0 und 1 ==> Angabe ist exklusiv !

# labels (Klassen)
label = ds.iloc[:,3].values

print("Daten wurden geholt und vorbereitet...")

#print("")
#ds
#data
#label

Daten wurden geholt und vorbereitet...


Ein wichtiger Punkt bei instanzbasierten Verfahren ist die Normalisierung. Als Vergleich habe ich hier eine Lösung mit Sklearn umgesetzt, welche ich jedoch nur zum Testen gebraucht haben.

In [ ]:
# Wert geteilt durch Max.
def normalizeDataSklearn(values):
  normalized_array = preprocessing.normalize(values, axis=0,norm="max")

  #print(normalized_array)
  return normalized_array

print("Funktion normalizeDataSklearn wurde erstellt...")


Funktion normalizeDataSklearn wurde erstellt...


Im folgenden ist meine eigene Methode, welche ich auch hier verwenden habe. Ich nutze heirbei globale Variablen, da ich nicht nur die Daten, sondern auch eine neue Instanz Normalisieren muss. Hierfür dient die Funktion normalizeSingleData, welch enur einen Wert normalisert im Gegensatz zu normalizeData.

In [ ]:
# Normalisiert zwischen 0 und 1
minValue=0           # ==> werden später gebraucht
maxValue=0           # ==> werden später gebraucht
max_minus_min=0      # ==> werden später gebraucht

def normalizeSingleData(value):
  return float((value-minValue)/max_minus_min)

def normalizeData(values):
  global minValue # ==> werden später gebraucht, siehe oben
  minValue=np.min(values)
  global maxValue # ==> werden später gebraucht, siehe oben
  maxValue=np.max(values)
  global max_minus_min # ==> werden später gebraucht, siehe oben
  max_minus_min=maxValue-minValue

  minValuex=minValue

  normalizedValues =np.empty(0)

  for curValue in values:
    normValue=float((curValue-minValue)/max_minus_min)
    normalizedValues=np.append(normalizedValues,normValue)

  #print(normalizedValues)
  return normalizedValues

print("Funktion normalizeData wurde erstellt...")

Funktion normalizeData wurde erstellt...


Ab hier fange ich an, die Logig für die Entfernungsberechnung zu bauen. Auch hier setze ich zunächst eine (interessante) Lösung von Sklearn an.

Wenn np.linalg.norm() auf einer array-ähnlichen Eingabe ohne zusätzliche Argumente aufgerufen wird, ist das Standardverhalten, die L2-Norm auf einer abgeflachten Ansicht des Arrays zu berechnen. Dies ist die Quadratwurzel aus der Summe der quadrierten Elemente und kann als die Länge des Vektors im euklidischen Raum interpretiert werden.

In [ ]:
#https://www.delftstack.com/de/api/numpy/python-numpy-numpy.linalg.norm-function/
#https://www.educative.io/edpresso/what-is-the-nplinalgnorm-method-in-numpy
#https://sparrow.dev/numpy-norm/

def calculateDistanceSklearn(instance1, instance2):
    # just in case, if the instances are lists or tuples:
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)
    
    return np.linalg.norm(instance1 - instance2)

#print(calculateDistanceSklearn([1, 4, 2, 5], [3, 7, 4, 1]))
print(calculateDistanceSklearn([1, 3], [2, 6]))
print("Funktion calculateDistanceSklearn wurde erstellt...")

3.1622776601683795
Funktion calculateDistanceSklearn wurde erstellt...


Um solch eine Funktion auch mal selbst umzusetzen, habe ich die Funktionalität für zwei dimensionale Daten mal selbst programmiert und verwende ihn auch für meine Lösung.

Beide algorithmen liefern heirbei anhand von Testdaten gleiche Ergebnisse.

In [ ]:
def calculateDistance2D(instance1, instance2):
  instance1 = np.array(instance1) 
  instance2 = np.array(instance2)

  v_ges=0

  for x in range(2):
    v_ges=v_ges + pow(instance1[x]-instance2[x],2)

  return math.sqrt(v_ges)

# x= calculateDistance2D([1,3],[2,6])
print(x)
print("Funktion calculateDistance wurde erstellt...")

3.1622776601683795
Funktion calculateDistance wurde erstellt...


Die folgende Zelle dient dafür, meinen Datenbestand zu normalisieren (anhand der weiter oben definierten Funktion).

Ab hier habe ich zwei Sätze mit normalisierten Daten: 

Einmal anhand meiner eigenen Funktion und einmal mit Hilfe von Sklearn. Auch hier will ich die Daten später zum Testen nutzen. Zudem ist der mit meinen Code normalisierte Satz um den label/Klasse erweitert. Dies macht den späteren Zugriff einfacher.

In [ ]:
x=np.array(data[:,0]).reshape(-1, 1)
y=np.array(data[:,1]).reshape(-1, 1)

data_norm_sklearn=normalizeDataSklearn(data)

data_norm_x=normalizeData(x)
data_norm_y=normalizeData(y)
data_norm=np.array([[x,y,l] for x,y,l in zip(data_norm_x,data_norm_y,label)])

#print(minValuex)
#print(maxValue)
#print(max_minus_min)
#print("")

print("Daten wurden normalisiert...")

Daten wurden normalisiert...


Hier kommt der Kern des Algorithmuses. Ihm wird eine zwei dimensionale Instanz übergeben und zudem ein k. Er prüft die neue Instanz an alle vorhandene Werte und liefert eine sortierte Liste der Entfernungen und Klassen der Länge k zurück.

In [ ]:
def get_k_nearest(newInstance2D, k):
  # Ich verzichte auf ein Splitten der Daten, da bei der hier vorgestellten Lösung 
  # eine neue Instanz immer gegen alle anderen getestet wird

  entfernung_klasse = []
  for x in data_norm:
    y= calculateDistance2D(x,newInstance2D)
    entfernung_klasse.append([y,x[2]])
    
  entfernung_klasse.sort(key=lambda x: x[0])
  return entfernung_klasse[:k+1]

print("Funktion get_k_nearest wurde erstellt...")

Funktion get_k_nearest wurde erstellt...


Die folgende Methode zählt mir die Anzahl der labels in meinen k-stelligen Ergebnis aus. Die umsetzung ist etwas tricky, dafür habe ich die Bibliothek/das Object defaultdict entdeckt und verwendet.

https://stackoverflow.com/questions/5900578/how-does-collections-defaultdict-work

 "Usually, a Python dictionary throws a KeyError if you try to get an item with a key that is not currently in the dictionary. The defaultdict in contrast will simply create any items that you try to access (provided of course they do not exist yet)." 

Mit Hilfe dieser Methode erhalte ich dynamische eine Auszählung der Instanzen zu den einzelnen Labeln. Diese muss nur sortiert werden und liefert dann das Label mit den meißten Einträgen/Nachbarn zurück.

In [ ]:
#https://stackoverflow.com/questions/5900578/how-does-collections-defaultdict-work

from collections import defaultdict

def get_label_count(neighbors):
  countList=[]
  
  d = defaultdict(list)
  for entfernung, label in neighbors:
    d[label].append(entfernung)
  
  best_label =[]
  for key in d:
    best_label.append([key,len(d[key])])
    
    #print(key)
    #print(d[key])
    #print(len(d[key]))
    #print("")

  best_label.sort(key=lambda x: x[1],reverse=True)

  print(best_label)

  return best_label[0]

print("Funktion get_label_count wurde erstellt...")


Funktion get_label_count wurde erstellt...


## Lösung der Aufgabe

Der folgende Code kann als Lösung der Aufgabe betrachtet werden. Er übergibt eine zuvor normailisierte Instanz an get_k_nearest. Das Ergebnis wird mit get_label_count ausgewerten und aufgeräumt ausgegeben.

Die Zuordnung dreier bekannter Instanzen war erfolgreich. Das Auftauchen gleicher Werte im Array ist mit dem vorhandensein gleicher Basisdaten zu erklären.

Natürlich währe die Lösung in einer IDE übersichtlicher und zudem das Problem mit wesendlich weniger Code lösbar. Aber ein Notebook ist ja auch ein Werkzeug, um sich Code und Daten flexible und dynamisch beschäftigen zu können.

In [ ]:
auftraege=normalizeSingleData(8)
erfolgreich=normalizeSingleData(7)

# 8/7 passt
#12/8 passt
# 4/4 passt

# 8 entspricht in etwa der Wurzel aus 59
# mit 8 kam es öfter mal zu Fehlern, 1 scheint die bessere Wahl
x=get_k_nearest([auftraege,erfolgreich],10)
y=get_label_count(x)

print("Gesamtbild: ")
print("")

print("Die 8 engsten Nachbarn sind: ")
print(x)

print("")

print("Die wahrscheinlichste Klasse ist: " + str(y[0]) + " mit " + str(y[1]) + " Mitgliedern")

[[2.0, 8], [1.0, 3]]
Gesamtbild: 

Die 8 engsten Nachbarn sind: 
[[0.03296703296703296, 2.0], [0.03296703296703296, 2.0], [0.03296703296703296, 2.0], [0.03296703296703296, 2.0], [0.03296703296703296, 2.0], [0.0786693465015184, 1.0], [0.0786693465015184, 1.0], [0.0786693465015184, 1.0], [0.0838699863872184, 2.0], [0.0838699863872184, 2.0], [0.0838699863872184, 2.0]]

Die wahrscheinlichste Klasse ist: 2.0 mit 8 Mitgliedern
